In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install pymongo
!pip install dnspython
!pip install pymongo[srv]
!pip install aiohttp
!pip install brotlipy

In [ ]:
import os
import time
import pymongo
import asyncio
import aiohttp
import requests
from bs4 import BeautifulSoup

In [ ]:
class Wnacg:
    def __init__(self):
        self.url = 'https://wnacg.org/photos-index-aid-99357.html'
        db = json.load(open('db.json', 'r', encoding='utf-8'))
        self.client = pymongo.MongoClient(db['db'])
        self.wnacg_db = self.client['Wnacg']
        self.wnacg_data_collect = self.wnacg_db['wnacg_data']
        self.wnacg_all_page_data_collect = self.wnacg_db['wnacg_all_page_data']
        self.wnacg_image_result_url_collect = self.wnacg_db['wnacg_image_result_url']
        # self.wnacg_image_result_url_collect.find().batch_size(60)
        self.headers = {
            'cookie': '__cfduid=d674bd993f1b071a94a3cefaa266e9ce81589795923',
            'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
        self.data = dict()
        self.last_page = 0
        self.path = '/gdrive/My Drive/Wnacg/'
        self.ok = 0
        self.ban = '//:*?"<>|.'
        self.temp_result_url = dict()
        self.skip = ['(C87) [グレートキャニオン (ディープバレー)] [人妻一航戦と種付け演習♪～赤城さんと加賀さんの豊満な船体目掛けてキモオタ珍宝急降下爆撃&精虫魚雷で卵子撃チン♪繁殖任務性交セリ～(艦隊これくしょん -艦これ-)][中国翻訳](98955)']
        self.error = False
    def badname(self, name):
        for i in self.ban:
            name = str(name).replace(i, ' ')
        return name.strip()

    def get_data_threading(self):
        for i in range(1, self.last_page + 1):
            print(i)
            url = f'https://wnacg.org/albums-index-page-{i}.html'
            self.get_data(url)
            if self.last_page // 2 == i:
                self.wnacg_data_collect.insert_one(self.data)
                self.data = dict()
        self.wnacg_data_collect.insert_one(self.data)

    def get_data(self, url=None):
        res = requests.get(url=url, headers=self.headers)
        if res.ok:
            res = res.text
            html = BeautifulSoup(res, features='lxml')
            for i in html.find_all('div', class_='pic_box'):
                num = i.find("a")["href"].split('.')[0].split('-')[-1]
                data_url = f'https://wnacg.org/{i.find("a")["href"]}'
                title = i.find('a')['title']
                for j in i.find('a'):
                    image = f'https:{j["src"]}'
                self.data.update({num: {'url': data_url,
                                        'image': image,
                                        'title': title}})
        else:
            self.get_data(url)

    def get_last_page(self):
        albums = 'https://wnacg.org/albums.html'
        res = requests.get(url=albums, headers=self.headers).text
        html = BeautifulSoup(res, features='lxml')
        for i, m in enumerate(html.find('div', class_='f_left paginator')):
            if i == 11:
                self.last_page = int(m.text)

    def view_data(self):
        data = dict()
        for m, i in enumerate(self.wnacg_data_collect.find({}, {'_id': False})):
            data.update(i)
        result = []
        for k, v in data.items():
            if f'{data[k]["title"]}({k})' not in result:
                result.append(f'{data[k]["title"]}({k})')
            else:
                print(data[k]['title'])

    def get_page_mission(self, run=False):
        if run:
            data = dict()
            for i, m in enumerate(self.wnacg_data_collect.find({}, {'_id': False})):
                data.update(m)
            result = []
            temp_page_data = []
            for i, m in enumerate(data):
                temp_page_data.append({f'{data[m]["title"]}({m})': data[m]['url']})
                if len(temp_page_data) == 24:
                    result.append(temp_page_data)
                    temp_page_data = []
                elif i == len(data) - 1:
                    result.append(temp_page_data)
            for i in result:
                loop = asyncio.get_event_loop()
                loop.run_until_complete(self.split_page_data(i))
                loop = None

    async def split_page_data(self, data):
        tasks = []
        for i in data:
            for k, v in i.items():
                tasks.append(asyncio.ensure_future(self.get_page_data(url=v, title=k)))
        if len(tasks) > 0:
            await asyncio.wait(tasks, timeout=3600)
            await asyncio.sleep(0)

    async def get_page_data(self, url, title):
        async with aiohttp.ClientSession() as session:
            async with session.get(url=url, headers=self.headers) as res:
                url_data = await res.text(encoding='utf-8', errors='ignore')
                data = dict()
                title = self.badname(title)
                num = url.split('aid-')[1].split('.')[0]
                html = BeautifulSoup(url_data, features='lxml')
                all_page = list()
                # image_url = {'1': []}
                start_page = 1
                end_page = 2
                for i in html.select("label"):
                    page = int(i.find_next_sibling().text.split('：')[1][:-1])
                    break
                if page // 12 > 0:
                    if page % 12 == 0:
                        end_page = page // 12 + 1
                    else:
                        end_page = page // 12 + 2
                for i in range(start_page, end_page):
                    page_url = f'https://wnacg.org/photos-index-page-{i}-aid-{num}.html'
                    all_page.append(page_url)
                data.update({title: all_page})
                self.wnacg_all_page_data_collect.insert_one(data)

    def get_image_url(self, run=False):
        if run:
            ok = []
            for i in self.wnacg_image_result_url_collect.find({}, {'_id': False}):
                ok.append(list(i.keys())[0])
            for i in self.wnacg_all_page_data_collect.find({}, {'_id': False}):
                key = list(i.keys())[0]
                value = list(i.values())[0]
                if key not in ok:
                    loop = asyncio.get_event_loop()
                    loop.run_until_complete(self.get_image_result_mission(url=value))
                    loop = None
                    data = {key: self.temp_result_url}
                    self.wnacg_image_result_url_collect.insert_one(data)
                    self.temp_result_url = dict()

    async def get_image_result_mission(self, url):
        tasks = []
        for i, m in enumerate(url):
            tasks.append(asyncio.ensure_future(self.get_image_result_url(url=m, num=i)))
        if len(tasks) > 0:
            await asyncio.wait(tasks, timeout=3600)
            await asyncio.sleep(0)

    async def get_image_result_url(self, url, num):
        async with aiohttp.ClientSession() as session:
            async with session.get(url=url, headers=self.headers) as res:
                url_data = await res.text(encoding='utf-8', errors='ignore')
                html = BeautifulSoup(url_data, features='lxml')
                for i, m in enumerate(html.find_all('div', class_='pic_box tb')):
                    self.temp_result_url.update({str(i + (num * 12) + 1): f'https://wnacg.org/{m.find("a")["href"]}'})

    def get_image(self):
        temp_image = []
        result = []
        data = dict()
        for i in self.wnacg_image_result_url_collect.find({}, {'_id': False}, no_cursor_timeout=True):
            data.update(i)
        count = 0
        mission_count = 11600  # 11687
        mission_go = 0
        for k, v in data.items():
            if self.error:
                # 81841
                sys.exit()
            if mission_go > mission_count:
                try:
                    if not os.path.isdir(f'{self.path}{k}'):
                        os.mkdir(f'{self.path}{k}')
                    for key, value in v.items():
                        download_path = f'{self.path}{k}/{key}.jpg'
                        if not os.path.isfile(f'{self.path}{k}/{key}.jpg'):
                            temp_image.append({download_path: value})
                            if len(temp_image) == 24:
                                result.append(temp_image)
                                temp_image = []
                            elif count == len(v) - 1:
                                result.append(temp_image)
                                temp_image = []
                        count += 1
                    # print(f'準備下載:{k}')
                    for j in result:
                        loop = asyncio.get_event_loop()
                        loop.run_until_complete(self.get_image_mission(title=k, data=j))
                        loop = None
                    result = []
                    # print(f'下載完成:{k}')
                except Exception as e:
                    # print(f'{"-"*10} 有問題  跳過{k} {"-"*10} {e}')
                    print(mission_go, e)
            else:
                mission_go += 1
                # print(mission_go)
           
    async def get_image_mission(self, title, data):
        tasks = []
        for i in data:
            for k, v in i.items():
                value = os.path.split(k)[1]
                download_path = f'{self.path}{title}/{value}'
                if not os.path.isfile(download_path):
                    tasks.append(asyncio.ensure_future(self.download_image(title=title, url=v, download_path=download_path)))
        if len(tasks) > 0:
            await asyncio.wait(tasks, timeout=3600)
            await asyncio.sleep(0)

    async def download_image(self, title, url, download_path):
        async with aiohttp.ClientSession() as session:
            async with session.get(url=url, headers=self.headers) as res:
                # print(f'準備下載: {title} {os.path.split(download_path)[1]}')
                url_data = await res.text(encoding='utf-8', errors='ignore')
                html = BeautifulSoup(url_data, features='lxml')
                for i in html.find_all('span', id='imgarea'):
                    for j in i.find('a'):
                        image_url = f'https:{j["src"]}'
            try:
                async with session.get(url=image_url, headers=self.headers) as result:
                    image = await result.read()
                    with open(download_path, 'wb') as f:
                        f.write(image)
            except:
                self.error = True
                print(title)
                # print(f'{title} {os.path.split(download_path)[1]}', '下載完成', '檔案大小:', f'{os.path.getsize(download_path) // 1024} KB')

if __name__ == '__main__':
    wnacg = Wnacg()
    # wnacg.get_last_page()
    # wnacg.get_data_threading()
    # wnacg.get_data()
    # wnacg.view_data()
    # wnacg.get_page_mission()
    # wnacg.get_image_url()
    wnacg.get_image()


11601 [Errno 22] Invalid argument: '/gdrive/My Drive/Wnacg/(C96) [カミナリネコ (えーたろー)] ぼくたちは催眠にかかって勉強どころじゃない (ぼくたちは勉強ができない)[本催眠應用專治優柔寡斷耳背重聽性無能男主一套只要998名額有限請各後宮作女主們儘速來電漢化](85078)'
11601 [Errno 22] Invalid argument: '/gdrive/My Drive/Wnacg/[本催眠應用專治優柔寡斷耳背重聽性無能男主一套只要998名額有限請各後宮作女主們儘速來電漢化] (C96) [カミナリネコ (えーたろー)] ぼくたちは催眠にかかって勉強どころじゃない (ぼくたちは勉強ができない) [中国翻訳](85033)'
11601 [Errno 22] Invalid argument: '/gdrive/My Drive/Wnacg/[無邪気漢化組] (C96) [んほぉおおおぉおおおおおお♥おっ♥おっ♥ (おおさわらさだお)] 潜入先で倫理観ゼロの犯罪者に変態アナル舐め奉仕させられた結果寝取られ公衆便器娼婦になっちゃった正義のY豚ちゃん本 (対魔忍ユキカゼ) [中国翻訳](85034)'
11601 [Errno 22] Invalid argument: '/gdrive/My Drive/Wnacg/[無邪気漢化組](C96) [んほぉおおおぉおおおおおお♥おっ♥おっ♥ (おおさわらさだお)] 潜入先で倫理観ゼロの犯罪者に変態アナル舐め奉仕させられた結果寝取られ公衆便器娼婦になっちゃった正義のY豚ちゃん本 (対魔忍ユキカゼ)(84937)'
11601 [Errno 22] Invalid argument: '/gdrive/My Drive/Wnacg/(C96) [barista (霧瀬ミツル)] 爆乳ハンターレイシア クズ男に騙されて遺跡でゴブリンたちに捕まりデカち〇ぽ強制イマラチオされデカ乳レ〇プにょほぉん駄目ぇ駄目ぇイキたくない干からびちゃうぅぅアクメ地獄のち悪堕ちするお話(84855)'
11601 [Errno 22] Invalid argument: '/gdrive/My Drive/Wnacg/[无毒汉化组](C92) [ユリルラリカ (小島紗、Laz

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
